# Modelo de Detección de Fraude

Durante el siguiente script se detallan e implementan los pasos para la creación de un modelo de detección de fraude a partir de características

In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("FraudDetection") \
    .getOrCreate()

# Se decidió usar el modulo de Spark ya que cuenta con una gran capacidad para manejar grandes volumenes de datos, además de contar
# con modelos pre-cargados con funciones que ayudan a su entrenamiento y evaluación de manera más amigable, concisa y rapida.

# Exploratory Data Analysis (EDA)

Nos encargamos de cargar los datos y vemos en que consisten y que columnas podrán ayudarnos a alimentar el modelo

In [2]:
# Se cargan los datos y se muestran las primeras filas del df
df = spark.read.csv('datos_fraud.csv', header=True, inferSchema=True)
df.show()

+--------------------+---------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------+
|      transaction_id|timestamp|amount|         variable_01|         variable_02|         variable_03|         variable_04|         variable_05|         variable_06|         variable_07|         variable_08|         variable_09|         variable_10|         variable_11|         variabl

In [3]:
# Vemos los tipos de datos de cada una de las columnas del dataframe
df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- amount: double (nullable = true)
 |-- variable_01: double (nullable = true)
 |-- variable_02: double (nullable = true)
 |-- variable_03: double (nullable = true)
 |-- variable_04: double (nullable = true)
 |-- variable_05: double (nullable = true)
 |-- variable_06: double (nullable = true)
 |-- variable_07: double (nullable = true)
 |-- variable_08: double (nullable = true)
 |-- variable_09: double (nullable = true)
 |-- variable_10: double (nullable = true)
 |-- variable_11: double (nullable = true)
 |-- variable_12: double (nullable = true)
 |-- variable_13: double (nullable = true)
 |-- variable_14: double (nullable = true)
 |-- variable_15: double (nullable = true)
 |-- variable_16: double (nullable = true)
 |-- variable_17: double (nullable = true)
 |-- variable_18: double (nullable = true)
 |-- variable_19: double (nullable = true)
 |-- variable_20: double (nullable = true)
 |-- varia

Como vimos en el output anterior, el dataframe esta compuesto por cinco tipos de columnas, de las cuales:

- transaction_id: se trata de un identificador que nos ayuda a diferencias cada conjunto de caracteristicas asociado a un registro
- time_stamp: para nuestro modelo, podemos omitir esta columna pues la hora y fecha en la que se realizo la transacción no afecta necesariamente para saber si es o no un fraude.
- amount: la cantidad de credito que se maneje puede ser importante para el analisis, pues a una cantidad mayor de monto podemos interpretar que podría existir mayor riesgo de fraude.
- variables 1  a la 32: variables propias de la transaccion que nos ayudan directamente como entrada para nuestro modelo. Al ser un ejercicio el significado independiente de cada una de estas variables no es de importancia.
- is_fraud: es la etiqueta con la que sabremos si la transacción es un fraude o no.

In [4]:
# Ver estadísticas descriptivas
df.describe().show()

+-------+--------------------+-----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|      transaction_id|        timestamp|            amount|         variable_01|         variable_02|         variable_03|         variable_04|         variable_05|         variable_06|         variable_07|         variable_08|    

In [4]:
# Verificar valores nulos
from pyspark.sql.functions import col, isnan, when, count

#No cuenta con valores nulos
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+---------+------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+
|transaction_id|timestamp|amount|variable_01|variable_02|variable_03|variable_04|variable_05|variable_06|variable_07|variable_08|variable_09|variable_10|variable_11|variable_12|variable_13|variable_14|variable_15|variable_16|variable_17|variable_18|variable_19|variable_20|variable_21|variable_22|variable_23|variable_24|variable_25|variable_26|variable_27|variable_28|variable_29|variable_30|variable_31|variable_32|is_fraud|
+--------------+---------+------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----

In [5]:
# Submuestreo aleatorio de la clase mayoritaria (transacciones legítimas)
n_fraud = df.filter(col('is_fraud') == 1).count()
n_legit = df.filter(col('is_fraud') == 0).count()

df_legit_subsampled = df.filter(col('is_fraud') == 0).sample(False, n_fraud / n_legit, seed=42)

# Filtrar las transacciones fraudulentas
df_fraud = df.filter(col('is_fraud') == 1)

# Combinar el subconjunto de transacciones legítimas submuestreado con las transacciones fraudulentas
df = df_legit_subsampled.union(df_fraud)

Como vemos, el paso de limpieza de datos podemos omitirlo, ya que nuestro dataset no cuenta con datos nulos o incorrectos. 

También omitimos la normalización de los datos.

In [6]:
from pyspark.ml.feature import VectorAssembler

# Seleccionar las columnas relevantes
feature_cols = [c for c in df.columns if c not in ['transaction_id', 'timestamp', 'amount', 'is_fraud']]

# Convertir las características en un vector, esto para que las características de cada registro se interpreten como entrada para nuestro
# modelo.
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)


In [7]:
# Seleccionar la variable objetivo y las características.
df = df.select(['transaction_id', 'features', 'is_fraud'])
df.show()

+--------------------+--------------------+--------+
|      transaction_id|            features|is_fraud|
+--------------------+--------------------+--------+
|448eab14ab3be1100...|[0.00768898551573...|       0|
|b3366af5dd24da698...|[-0.0094397190234...|       0|
|f64640528d1bc4d50...|[-0.0647913465486...|       0|
|d69b16d989f997c4e...|[-0.1253158886478...|       0|
|82f6b02e113e27b6a...|[-0.0436232923044...|       0|
|2b58a1b1a4353a944...|[-0.0157642212306...|       0|
|c655ea515a0ecd105...|[0.15908081825467...|       0|
|8196944079c991dea...|[-4.0378445491197...|       0|
|924aaf66f8fa0fc12...|[-0.0390045305879...|       0|
|eed3cd952acca5e88...|[-0.0502841522394...|       0|
|a839296313a9dce64...|[1.61197545286133...|       0|
|e036495d9ed4b0cce...|[-0.1928334987314...|       0|
|84b31c379b7bbdc7d...|[-0.0490034311154...|       0|
|844c813a2822c32e1...|[0.01835038470872...|       0|
|7bb47998ec31c4aeb...|[0.06333697813373...|       0|
|4945e4cdd8d706739...|[0.21843694489386...|   

In [8]:
from pyspark.ml.classification import RandomForestClassifier

# Dividir los datos en conjuntos de entrenamiento y prueba.
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

# Entrenar el modelo
rf = RandomForestClassifier(labelCol="is_fraud", featuresCol="features", numTrees=100) 
model = rf.fit(train_df) # pySpark nos ayuda con la función fit para el entrenamiento automático del modelo

# Realizamos las  predicciones
predictions = model.transform(test_df)
predictions.select("transaction_id","features", "is_fraud", "prediction", "probability").show()


+--------------------+--------------------+--------+----------+--------------------+
|      transaction_id|            features|is_fraud|prediction|         probability|
+--------------------+--------------------+--------+----------+--------------------+
|45ac3e765206b042e...|[0.04247881396028...|       0|       0.0|[0.81101358334178...|
|8196944079c991dea...|[-4.0378445491197...|       0|       0.0|[0.80985467643012...|
|844c813a2822c32e1...|[0.01835038470872...|       0|       0.0|[0.90896582052891...|
|84b31c379b7bbdc7d...|[-0.0490034311154...|       0|       0.0|[0.93470861399586...|
|b65f6ee432897b062...|[0.06871696237983...|       0|       0.0|[0.89613758964788...|
|c655ea515a0ecd105...|[0.15908081825467...|       0|       0.0|[0.92193469849784...|
|d69b16d989f997c4e...|[-0.1253158886478...|       0|       0.0|[0.88671113307034...|
|0a113827fdc12f5af...|[0.30692452723109...|       0|       0.0|[0.94524271487312...|
|3a5bded1fad7cb8ae...|[0.04130870504749...|       0|       0.0|[0

# Testing de modelo
Además de la medida 

In [9]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# ROC AUC Score
evaluator = BinaryClassificationEvaluator(labelCol="is_fraud", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f'ROC AUC Score: {roc_auc}')

# Matriz de confusión
predictions.groupBy("is_fraud", "prediction").count().show()

# Precision y Recall
evaluator_precision = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="precisionByLabel")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="recallByLabel")
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# F1 Score
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f'F1 Score: {f1_score}')

# Accuracy
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f'Accuracy: {accuracy}')

ROC AUC Score: 0.967850331486695
+--------+----------+-----+
|is_fraud|prediction|count|
+--------+----------+-----+
|       0|       0.0|  141|
|       0|       1.0|    2|
|       1|       0.0|   25|
|       1|       1.0|  129|
+--------+----------+-----+

Precision: 0.8493975903614458
Recall: 0.986013986013986
F1 Score: 0.9088059955714529
Accuracy: 0.9090909090909091


In [10]:
# Convertir predicciones a un DataFrame de pandas
predictions_pd = predictions.select("transaction_id", "probability").toPandas()
predictions_pd['score'] = predictions_pd['probability'].apply(lambda x: x[1])
#El score representa así, la posibilidad de que la transacción sea fraudulenta
predictions_pd = predictions_pd.drop('probability', axis=1)

# Guardar como CSV
predictions_pd.to_csv('predicciones_fraude.csv', index=False)
